# Inspection of Files


In [ ]:
import sys
import fsspec
import xarray as xr
import hvplot.xarray
import zarr

In [ ]:
print("Python : ", sys.version)
print("fsspec : ", fsspec.__version__)
print("xarray : ", xr.__version__)
print("zarr   : ", zarr.__version__)

In [ ]:
fs = fsspec.filesystem('s3', anon=True)

In [ ]:
flist = fs.ls('s3://noaa-nwm-retrospective-2-1-pds/')
flist

In [ ]:
flist = fs.glob('noaa-nwm-retrospective-2-1-pds/model_output/*')
print(flist[0])
print(flist[-1])

In [ ]:
flist = fs.glob('noaa-nwm-retrospective-2-1-pds/model_output/1979/*LDAS*')
flist[0]

In [ ]:
flist = fs.glob('noaa-nwm-retrospective-2-1-pds/model_output/2020/*LDAS*')
flist[-1]

Okay, so at this point we've learned that we have 3-hourly output over roughly 40 years

In [ ]:
# %%time
# flist = fs.glob('noaa-nwm-retrospective-2-1-pds/model_output/*/*LDAS*')   # this is slow
40 * 365 * 24 / 3

In [ ]:
flist[0]

So about 117,000 NetCDF files! 

Let's check one out.  Although it's not super efficient, we can open a NetCDF file on S3 as a virtual file object with `fs.open(s3_url_of_netcdf_file)`.  If we open a dataset in xarray using `chunks=` we are telling xarray to use Dask, and `chunks={}` means use the native chunking in the NetCDF file

In [ ]:
ds = xr.open_dataset(fs.open(flist[0]), chunks={})

In [ ]:
ds

In [ ]:
ds.data_vars

In [ ]:
ds = ds[['ACCET', 'SNEQV', 'FSNO']]

In [ ]:
ds.data_vars

In [ ]:
ds['ACCET']

In [ ]:
ds.ACCET

The data is chunked as full spatial domain and 1 time step, with about 135MB chunk size.   This is actually great for visualization of maps at specific time steps or for calculations that involve the entire dataset. So kerchunking this data would be a nice first step. 

In [ ]:
%%time
da = ds.ACCET.load()

In [ ]:
da

In [ ]:
da.hvplot(x='x', y='y', rasterize=True, cmap='turbo', data_aspect=1)